# Counting bytes

The data
 1. Graph with relations between users:
```
uId1 {(fId1,mask1), (fId2,mask2),...}
```
 2. Demography data for the users:
```
uId createAt bornAt gender country location region is_core
```



## Step 1 - take a look at the data

In [1]:
ls ./

CountinfBytes.ipynb              results-20151009-103553.json
EY.key                           results-20151009-103553.parquet/
countingCommonFriends.ipynb      spark-warehouse/
derby.log                        spark_tutorial.ipynb
graph/                           trainDemography/
graph.zip                        trainDemography.zip
metastore_db/                    users


In [2]:
!du -Lh ./*

132K	./CountinfBytes.ipynb
3.7G	./graph
3.7G	./graph.zip
3.9M	./results-20151009-103553.json
1.6M	./results-20151009-103553.parquet
  0B	./spark-warehouse
152K	./spark_tutorial.ipynb
2.4G	./trainDemography
2.4G	./trainDemography.zip
372K	./users


In [6]:
!cat ./graph/part* | wc -l

  475231


In [11]:
!cat ./trainDemography/part* | wc -l

 47245400


In [12]:
!cat ./users | wc -l

   43110


### Inside demography

In [3]:
!head ./trainDemography/part-00000

7	1371193965320	2683	2	10424076448	3400671		0
23	1353343917370	7179	2	10414533690	3615719		0
39	1222604396110	-2760	1	10414533690	3538869	36	0
55	1316262667460	-3822	2	10414533690	2116310	0	0
71	1330702003717	7469	1	10414533690	1467002		0
87	1347519172347	5732	1	10410450732	2376681	20	0
103	1361238817847	3896	2	10414533690	3385314		0
119	1336812119450	7272	2	10414533690	917969		0
135	1253287173093	4477	2	10411801535	3501686	15	0
151	1258894487967	7611	1	10397135919	1938023	0	0


### Inside graph

In [4]:
!head ./graph/part-00000

423	{(76034,0),(91316,0),(221981,768),(459889,0),(646088,0),(818298,0),(1095140,256),(1268621,0),(1448296,0),(1740815,512),(2027913,512),(2138857,0),(2385713,0),(2418739,0),(2530917,0),(2635510,0),(2826341,16384),(2839495,0),(2989934,0),(3089877,0),(3129699,0),(3401392,0),(3404580,0),(3691841,0),(3818915,0),(3961671,0),(3962590,0),(4361197,0),(4603978,0),(4630772,0),(4635165,0),(4741186,0),(5123399,768),(5144484,0),(5208606,256),(5260990,0),(5726151,0),(5732940,0),(5865006,0),(5882540,0),(6079227,0),(6541710,0),(6564764,0),(6796989,256),(6806586,0),(6882410,0),(6919672,0),(6955914,0),(7016436,0),(7050665,0),(7127474,0),(7315932,0),(7404850,0),(7539771,256),(7568145,0),(7613173,0),(7814624,0),(7854615,0),(7938003,0),(8039078,0),(8116826,512),(8408934,0),(8509468,0),(8940623,0),(8945833,0),(8985804,0),(9029424,0),(9251197,0),(9428947,0),(9789698,0),(9965397,0),(10239205,512),(10326334,512),(10572767,0),(10584590,0),(10872286,0),(10961676,1024),(11215787,0),(11246356,0),(11874873,512),(11

## Step 2 - preparing environment

In [1]:
import csv, os                      # Reading/writing files
import resource, timeit             # Measuring resources
from pympler import asizeof         # Measuring object size
from collections import defaultdict # Simpler collections initialization

In [2]:
# Prints time taken from the start and current memory usage
def print_resource_usage(startTime):
    print "Time passed %s (seconds)" % str(timeit.default_timer() - startTime)
    print "Memory usage overal: %s (mb)" % (
        resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / (1024 * 1024))

In [3]:
print_resource_usage(timeit.default_timer())

Time passed 1.90734863281e-06 (seconds)
Memory usage overal: 36 (mb)


In [4]:
# Data paths
dataPath = "./"
graphPath = os.path.join(dataPath, "graph")
demographyPath = os.path.join(dataPath, "trainDemography")
testUsersPath = os.path.join(dataPath, "users")
resultsPath = "./results"

## Step 3 - Extract/Transfrom/Load (ETL)

### Loading 1 / 16 graph

In [6]:
start = timeit.default_timer()

mineFriends = defaultdict(dict)

# Iterate all files in a graph path
for file in [f for f in os.listdir(graphPath) if f.startswith("part-00000")]:
    # Now iterate over lines in each file
    for line in csv.reader(open(os.path.join(graphPath, file)),
                            delimiter='\t'):
        user = int(line[0])

        # Split each line into user and his friends
        for friendship in line[1][2:len(line[1]) - 2].split("),("):
            parts=friendship.split(",")
            friend = int(parts[0])
            mask = int(parts[1])
            mineFriends[user][friend] = mask
print_resource_usage(start)

Time passed 59.1145188808 (seconds)
Memory usage overal: 1614 (mb)


### Loading demography

In [7]:
start = timeit.default_timer()

birthDates = dict()
# Iterate all files in demography path
for file in [f for f in os.listdir(demographyPath) if f.startswith("part")]:
    csv_reader = csv.reader(open(os.path.join(demographyPath, file)), 
                            delimiter='\t')
    # Extract birth date from each line
    for line in csv_reader:
        user = int(line[0])
        birthDates[user] = int(line[2]) if line[2] != '' else 0
            
print_resource_usage(start)

Time passed 215.427893877 (seconds)
Memory usage overal: 4162 (mb)


#### Something went wrong
* Data is 6 gigabytes of uncompressed texts on disk
* But it takes already 4.4 gigabytes in RAM although highly filtered (1/16 of graph data, only one column from demography)...

#### Why this happens?
* Inefficient data structures
* Boxing

## Hash-table with linked lists

<img src="https://upload.wikimedia.org/wikipedia/commons/d/d0/Hash_table_5_0_1_1_1_1_1_LL.svg" width="800"/>

## Hash-table with open addressing

<img src="https://upload.wikimedia.org/wikipedia/commons/b/bf/Hash_table_5_0_1_1_1_1_0_SP.svg" width="800" />

## Lets look inside
```c
#define PyDict_MINSIZE 8
typedef struct _dictobject PyDictObject;
struct _dictobject {
  ...
  PyDictEntry *ma_table;
  ...
};

typedef struct {
  Py_ssize_t me_hash;
  PyObject *me_key;
  PyObject *me_value;
} PyDictEntry;
```

### Important notes

* Dictionary is an array of objects
* This array is at least 1/3 empty
* Each array entry stores 2 pointers and hash (2 * 8 + 4 = 20 bytes)
* Dictionary with 47 millions of records uses at least 47M \* 4 / 3 \* 20 = 1253M bytes
  * 3500M bytes at worse
* Data are *boxed* and *data locality* is broken

### Data locality - why it matters?

<center>
<img src="https://s3.amazonaws.com/media-p.slid.es/uploads/71982/images/2310246/figure04-caching-pyramid.png" width="600"/>
</center>

| Source   | Latency       |
| ---------|:-------------:|
| L1 Cache |  1-2 ns       |
| L2 Cache |  3-5 ns       |
| L3 Cache |  10-40 ns     |
| DRAM     |  60-100 ns    |



## Boxing at a glance
```c
#define PyObject_HEAD                   \
    _PyObject_HEAD_EXTRA                \
    Py_ssize_t ob_refcnt;               \
    struct _typeobject *ob_type;

typedef struct {
    PyObject_HEAD
    long ob_ival;
} PyIntObject;
```

### Bad news

* Putting integer in a box adds 16 bytes of overhead (20 bytes in total)
* Dictionary with 47 millions of int-int pairs uses about 1253M + 47M \* 20 \* 2 = 3133M bytes

# Good news
* With efficient data structure we could expect 47M \* 8 = 376M bytes
* If we are luky, even 47M \* 4 = 188M bytes is possible

## Choosing a data structure - Dictionary

* Pros
  + Very simple and convinient
* Cons
  - Too much of memory overhead when used for primitive types
  - Data are stored in random memory areas - L3/L2/L1 caches can not work efficiently

## Choosing a data structure - NumPy arrays

* Pros
  + No memory overhead 
  + No boxing
  + Data are sequential in memory
* Cons
  - Require keys to be nearly sequential
  - Require size before initialization


#### Best choise for demography, but won't work for graph representation

## Choosing a data structure - scipy sparse matrices

+ Suited for storing large matrices with a lot of zeros (sparse)
+ Backed by numpy arrays
+ Different formats

### Sparse matrices types
* COO - three arrays with indicies (column and row) and data.
 * Simplest for construction, well integrated with Pandas.
* CSR - one small array with monotonically increasing start index for each row and two arrays for column indices and values. 
 * Less memory footprint, fast operations.
* CSC - same as CSR, but with column indices in the first array.

#### CSR matrix
<img src="http://hamukazu.com/wp-content/uploads/2014/12/csr_matrix.png" width="800"/>

#### CSC matrix
<img src="http://hamukazu.com/wp-content/uploads/2014/12/csc_matrix2.png" width="800"/>



#### Best choise for graph representation

## Lets do it right this time

#### Calculate graph statistics

In [5]:
start = timeit.default_timer()
maxLeft = 0
maxTotal = 0
linksCount = 0

print_resource_usage(start)

# Iterate all files in a graph path
for file in [f for f in os.listdir(graphPath) if f.startswith("part-00000")]:
    # Now iterate over lines in each file
    for line in csv.reader(open(os.path.join(graphPath, file)),
                            delimiter='\t'):
        user = int(line[0])
        maxLeft = max(maxLeft, user)

        # Split each line into user and his friends
        for friendship in line[1][2:len(line[1]) - 2].split("),("):
            linksCount += 1
            parts = friendship.split(",")
            friend = int(parts[0])
            maxTotal = max(maxTotal, friend)

maxTotal = max(maxTotal, maxLeft)        

print "max id %s at the left, max id %s in total, %s links" % (
    maxLeft, maxTotal, linksCount)

Time passed 0.00019097328186 (seconds)
Memory usage overal: 36 (mb)
max id 47288167 at the left, max id 47289241 in total, 18688874 links


#### Import data structures

In [6]:
import numpy
import scipy
from scipy.sparse import coo_matrix, csr_matrix

### Loading the graph as COO matrix (three arrays)

In [7]:
start = timeit.default_timer()
# Arrays with data
fromUser = numpy.zeros(linksCount, dtype=numpy.int32) 
toUser = numpy.zeros(linksCount, dtype=numpy.int32) 
masks = numpy.ones(linksCount, dtype=numpy.int32) 
current = 0  # Pointer to the current record to insert
for file in [f for f in os.listdir(graphPath) if f.startswith("part-00000")]:
    for line in csv.reader(open(os.path.join(graphPath, file)), 
                           delimiter='\t'):
        user = int(line[0])

        for friendship in line[1][2:len(line[1]) - 2].split("),("):
            parts=friendship.split(",")
            fromUser[current] = user - 1
            toUser[current] = int(parts[0]) - 1
            current += 1            

print_resource_usage(start)
print "Memory used by COO graph: %s (mb)" % (
    (fromUser.nbytes + toUser.nbytes + masks.nbytes) / (1024 * 1024))

Time passed 52.2690019608 (seconds)
Memory usage overal: 260 (mb)
Memory used by COO graph: 213 (mb)


#### Now transform arrays to the CSR matrix

In [8]:
def print_csr_size(matrix):
    print "Size of CSR matrix in memory is %s (mb)" % ((matrix.data.nbytes + matrix.indptr.nbytes + matrix.indices.nbytes)/ (1024 * 1024))

In [9]:
start = timeit.default_timer()

testGraph = coo_matrix(
    (masks, (fromUser, toUser)),
    shape=(maxTotal, maxTotal)).tocsr()

# These arrays are no longer needed, clean up:
del fromUser
del toUser
del masks

print_resource_usage(start)
print_csr_size(testGraph)

Time passed 10.4435501099 (seconds)
Memory usage overal: 884 (mb)
Size of CSR matrix in memory is 322 (mb)


#### Matrix multiplication? 

<img src="https://www.mathsisfun.com/algebra/images/matrix-multiply-a.gif" width="800"/>

In [11]:
start = timeit.default_timer()

commonFriends = testGraph.dot(testGraph)

print_resource_usage(start)

Time passed 59.1219229698 (seconds)
Memory usage overal: 2763 (mb)


## Conclusions

* Python indeed helps us to easy and fast *write* data analysis code :)
* But this code, if written in a "dummy" way, *works* slowly :(
* Fortunatelly, there are libs and tehcniques to do the same work in Python faster without too much complexity :)

# Keep calm, and keep counting bytes